#### Drew Mini-Project

_February 2022_

The purpose of this notebook is to explore several questions relating to the characteristics of audio in different acquisition systems / zones / file formats.

In [ ]:
# used to play audio in browser
import IPython.display as ipd

In [ ]:
# Note: paths will likely need to be rewritten depending on location from which notebook is run

# Zone 1 (farther - downtown)
OLD1_RECORDING_PATH = "/data/Zone1/2018_08_15/201808142342-953683-27730.mp3"
NEW1_RECORDING_PATH = "/data/Zone1/2021_11_13/20211113_02-49-23-C 107.2 TO 06-51-46.mp3"

# try Zone 3 (close - Woodlawn)
OLD3_RECORDING_PATH = "/data/Zone3/2018_08_15/201808142347-288991-14545.mp3"
NEW3_RECORDING_PATH = "/data/Zone3/2021_11_13/20211113_02-04-38-C 110.9 TO 04-57-17.mp3"

Old (method) recording path - Zone 1: 
- First 10 seconds silence
- With 30:12 left (20 seconds in), there's 10+ seconds of speech.

Old (method) recording path - Zone 3:
- First 10 seconds silence
- With 30:12 left (19 seconds in), there's 10+ seconds of speech.


New (method) recording path - Zone 1 and 3:
- This new recording method certainly is running noise activity detection. Very little silence.


In [ ]:
import torch
import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T
import librosa

print(torch.__version__)
print(torchaudio.__version__)

Get helper functions:

In [ ]:

#-------------------------------------------------------------------------------
# Preparation of data and helper functions.
#-------------------------------------------------------------------------------
import io
import os
import math
import tarfile
import multiprocessing

import scipy
import librosa
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import requests
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import time
from IPython.display import Audio, display

[width, height] = matplotlib.rcParams['figure.figsize']
if width < 10:
  matplotlib.rcParams['figure.figsize'] = [width * 2.5, height]

#### Information to fetch sample data

# _SAMPLE_DIR = "_sample_data"
# SAMPLE_WAV_URL = "https://pytorch-tutorial-assets.s3.amazonaws.com/steam-train-whistle-daniel_simon.wav"
# SAMPLE_WAV_PATH = os.path.join(_SAMPLE_DIR, "steam.wav")

# SAMPLE_WAV_SPEECH_URL = "https://pytorch-tutorial-assets.s3.amazonaws.com/VOiCES_devkit/source-16k/train/sp0307/Lab41-SRI-VOiCES-src-sp0307-ch127535-sg0042.wav"
# SAMPLE_WAV_SPEECH_PATH = os.path.join(_SAMPLE_DIR, "speech.wav")

# SAMPLE_RIR_URL = "https://pytorch-tutorial-assets.s3.amazonaws.com/VOiCES_devkit/distant-16k/room-response/rm1/impulse/Lab41-SRI-VOiCES-rm1-impulse-mc01-stu-clo.wav"
# SAMPLE_RIR_PATH = os.path.join(_SAMPLE_DIR, "rir.wav")

# SAMPLE_NOISE_URL = "https://pytorch-tutorial-assets.s3.amazonaws.com/VOiCES_devkit/distant-16k/distractors/rm1/babb/Lab41-SRI-VOiCES-rm1-babb-mc01-stu-clo.wav"
# SAMPLE_NOISE_PATH = os.path.join(_SAMPLE_DIR, "bg.wav")

# SAMPLE_MP3_URL = "https://pytorch-tutorial-assets.s3.amazonaws.com/steam-train-whistle-daniel_simon.mp3"
# SAMPLE_MP3_PATH = os.path.join(_SAMPLE_DIR, "steam.mp3")

# SAMPLE_GSM_URL = "https://pytorch-tutorial-assets.s3.amazonaws.com/steam-train-whistle-daniel_simon.gsm"
# SAMPLE_GSM_PATH = os.path.join(_SAMPLE_DIR, "steam.gsm")

# SAMPLE_TAR_URL = "https://pytorch-tutorial-assets.s3.amazonaws.com/VOiCES_devkit.tar.gz"
# SAMPLE_TAR_PATH = os.path.join(_SAMPLE_DIR, "sample.tar.gz")
# SAMPLE_TAR_ITEM = "VOiCES_devkit/source-16k/train/sp0307/Lab41-SRI-VOiCES-src-sp0307-ch127535-sg0042.wav"

# S3_BUCKET = "pytorch-tutorial-assets"
# S3_KEY = "VOiCES_devkit/source-16k/train/sp0307/Lab41-SRI-VOiCES-src-sp0307-ch127535-sg0042.wav"

# YESNO_DATASET_PATH = os.path.join(_SAMPLE_DIR, "yes_no")
# os.makedirs(YESNO_DATASET_PATH, exist_ok=True)
# os.makedirs(_SAMPLE_DIR, exist_ok=True)

### Download sample data

# def _fetch_data():
#   uri = [
#     (SAMPLE_WAV_URL, SAMPLE_WAV_PATH),
#     (SAMPLE_WAV_SPEECH_URL, SAMPLE_WAV_SPEECH_PATH),
#     (SAMPLE_RIR_URL, SAMPLE_RIR_PATH),
#     (SAMPLE_NOISE_URL, SAMPLE_NOISE_PATH),
#     (SAMPLE_MP3_URL, SAMPLE_MP3_PATH),
#     (SAMPLE_GSM_URL, SAMPLE_GSM_PATH),
#     (SAMPLE_TAR_URL, SAMPLE_TAR_PATH),
#   ]
#   for url, path in uri:
#     with open(path, 'wb') as file_:
#       file_.write(requests.get(url).content)

# _fetch_data()

# def _download_yesno():
#   if os.path.exists(os.path.join(YESNO_DATASET_PATH, "waves_yesno.tar.gz")):
#     return
#   torchaudio.datasets.YESNO(root=YESNO_DATASET_PATH, download=True)

# YESNO_DOWNLOAD_PROCESS = multiprocessing.Process(target=_download_yesno)
# YESNO_DOWNLOAD_PROCESS.start()

### This appears to be used to fetch example samples exclusively, but I'll leave in just in case of wider applicability.

def _get_sample(path, resample=None):
  effects = [
    ["remix", "1"]  # remix merges channels to 1
  ]
  if resample:  # resample converts sample rate via interpolation
    effects.extend([
      ["lowpass", f"{resample // 2}"],
      ["rate", f'{resample}'],
    ])
  return torchaudio.sox_effects.apply_effects_file(path, effects=effects)

### More functions to get specific samples

# def get_speech_sample(*, resample=None):
#   return _get_sample(SAMPLE_WAV_SPEECH_PATH, resample=resample)

# def get_sample(*, resample=None):
#   return _get_sample(SAMPLE_WAV_PATH, resample=resample)

# def get_rir_sample(*, resample=None, processed=False):
#   rir_raw, sample_rate = _get_sample(SAMPLE_RIR_PATH, resample=resample)
#   if not processed:
#     return rir_raw, sample_rate
#   rir = rir_raw[:, int(sample_rate*1.01):int(sample_rate*1.3)]
#   rir = rir / torch.norm(rir, p=2)
#   rir = torch.flip(rir, [1])
#   return rir, sample_rate

# def get_noise_sample(*, resample=None):
#   return _get_sample(SAMPLE_NOISE_PATH, resample=resample)

def print_stats(waveform, sample_rate=None, src=None):
  if src:
    print("-" * 10)
    print("Source:", src)
    print("-" * 10)
  if sample_rate:
    print("Sample Rate:", sample_rate)
  print("Shape:", tuple(waveform.shape))
  print("Dtype:", waveform.dtype)
  print(f" - Max:     {waveform.max().item():6.3f}")
  print(f" - Min:     {waveform.min().item():6.3f}")
  print(f" - Mean:    {waveform.mean().item():6.3f}")
  print(f" - Std Dev: {waveform.std().item():6.3f}")
  print()
  print(waveform)
  print()

def plot_waveform(waveform, sample_rate, title="Waveform", xlim=None, ylim=None):
  waveform = waveform.numpy()

  num_channels, num_frames = waveform.shape
  time_axis = torch.arange(0, num_frames) / sample_rate

  figure, axes = plt.subplots(num_channels, 1)
  if num_channels == 1:
    axes = [axes]
  for c in range(num_channels):
    axes[c].plot(time_axis, waveform[c], linewidth=1)
    axes[c].grid(True)
    if num_channels > 1:
      axes[c].set_ylabel(f'Channel {c+1}')
    if xlim:
      axes[c].set_xlim(xlim)
    if ylim:
      axes[c].set_ylim(ylim)
  figure.suptitle(title)
  plt.show(block=False)

def plot_specgram(waveform, sample_rate, title="Spectrogram", xlim=None):
  waveform = waveform.numpy()

  num_channels, num_frames = waveform.shape
  time_axis = torch.arange(0, num_frames) / sample_rate

  figure, axes = plt.subplots(num_channels, 1)
  if num_channels == 1:
    axes = [axes]
  for c in range(num_channels):
    axes[c].specgram(waveform[c], Fs=sample_rate)
    if num_channels > 1:
      axes[c].set_ylabel(f'Channel {c+1}')
    if xlim:
      axes[c].set_xlim(xlim)
  figure.suptitle(title)
  plt.show(block=False)

def play_audio(waveform, sample_rate):
  waveform = waveform.numpy()

  num_channels, num_frames = waveform.shape
  if num_channels == 1:
    display(Audio(waveform[0], rate=sample_rate))
  elif num_channels == 2:
    display(Audio((waveform[0], waveform[1]), rate=sample_rate))
  else:
    raise ValueError("Waveform with more than 2 channels are not supported.")

def inspect_file(path):
  print("-" * 10)
  print("Source:", path)
  print("-" * 10)
  print(f" - File size: {os.path.getsize(path)} bytes")
  print(f" - {torchaudio.info(path)}")

def plot_spectrogram(spec, title=None, ylabel='freq_bin', aspect='auto', xmax=None):
  fig, axs = plt.subplots(1, 1)
  axs.set_title(title or 'Spectrogram (db)')
  axs.set_ylabel(ylabel)
  axs.set_xlabel('frame')
  im = axs.imshow(librosa.power_to_db(spec), origin='lower', aspect=aspect)
  if xmax:
    axs.set_xlim((0, xmax))
  fig.colorbar(im, ax=axs)
  plt.show(block=False)

def plot_mel_fbank(fbank, title=None):
  fig, axs = plt.subplots(1, 1)
  axs.set_title(title or 'Filter bank')
  axs.imshow(fbank, aspect='auto')
  axs.set_ylabel('frequency bin')
  axs.set_xlabel('mel bin')
  plt.show(block=False)

### This is just an example

# def get_spectrogram(
#     n_fft = 400,
#     win_len = None,
#     hop_len = None,
#     power = 2.0,
# ):
#   waveform, _ = get_speech_sample()
#   spectrogram = T.Spectrogram(
#       n_fft=n_fft,
#       win_length=win_len,
#       hop_length=hop_len,
#       center=True,
#       pad_mode="reflect",
#       power=power,
#   )
#   return spectrogram(waveform)

def plot_pitch(waveform, sample_rate, pitch):
  figure, axis = plt.subplots(1, 1)
  axis.set_title("Pitch Feature")
  axis.grid(True)

  end_time = waveform.shape[1] / sample_rate
  time_axis = torch.linspace(0, end_time,  waveform.shape[1])
  axis.plot(time_axis, waveform[0], linewidth=1, color='gray', alpha=0.3)

  axis2 = axis.twinx()
  time_axis = torch.linspace(0, end_time, pitch.shape[1])
  ln2 = axis2.plot(
      time_axis, pitch[0], linewidth=2, label='Pitch', color='green')

  axis2.legend(loc=0)
  plt.show(block=False)

def plot_kaldi_pitch(waveform, sample_rate, pitch, nfcc):
  figure, axis = plt.subplots(1, 1)
  axis.set_title("Kaldi Pitch Feature")
  axis.grid(True)

  end_time = waveform.shape[1] / sample_rate
  time_axis = torch.linspace(0, end_time,  waveform.shape[1])
  axis.plot(time_axis, waveform[0], linewidth=1, color='gray', alpha=0.3)

  time_axis = torch.linspace(0, end_time, pitch.shape[1])
  ln1 = axis.plot(time_axis, pitch[0], linewidth=2, label='Pitch', color='green')
  axis.set_ylim((-1.3, 1.3))

  axis2 = axis.twinx()
  time_axis = torch.linspace(0, end_time, nfcc.shape[1])
  ln2 = axis2.plot(
      time_axis, nfcc[0], linewidth=2, label='NFCC', color='blue', linestyle='--')

  lns = ln1 + ln2
  labels = [l.get_label() for l in lns]
  axis.legend(lns, labels, loc=0)
  plt.show(block=False)

DEFAULT_OFFSET = 201
SWEEP_MAX_SAMPLE_RATE = 48000
DEFAULT_LOWPASS_FILTER_WIDTH = 6
DEFAULT_ROLLOFF = 0.99
DEFAULT_RESAMPLING_METHOD = 'sinc_interpolation'

def _get_log_freq(sample_rate, max_sweep_rate, offset):
  """Get freqs evenly spaced out in log-scale, between [0, max_sweep_rate // 2]

  offset is used to avoid negative infinity `log(offset + x)`.

  """
  half = sample_rate // 2
  start, stop = math.log(offset), math.log(offset + max_sweep_rate // 2)
  return torch.exp(torch.linspace(start, stop, sample_rate, dtype=torch.double)) - offset

def _get_inverse_log_freq(freq, sample_rate, offset):
  """Find the time where the given frequency is given by _get_log_freq"""
  half = sample_rate // 2
  return sample_rate * (math.log(1 + freq / offset) / math.log(1 + half / offset))

def _get_freq_ticks(sample_rate, offset, f_max):
  # Given the original sample rate used for generating the sweep,
  # find the x-axis value where the log-scale major frequency values fall in
  time, freq = [], []
  for exp in range(2, 5):
    for v in range(1, 10):
      f = v * 10 ** exp
      if f < sample_rate // 2:
        t = _get_inverse_log_freq(f, sample_rate, offset) / sample_rate
        time.append(t)
        freq.append(f)
  t_max = _get_inverse_log_freq(f_max, sample_rate, offset) / sample_rate
  time.append(t_max)
  freq.append(f_max)
  return time, freq

def plot_sweep(waveform, sample_rate, title, max_sweep_rate=SWEEP_MAX_SAMPLE_RATE, offset=DEFAULT_OFFSET):
  x_ticks = [100, 500, 1000, 5000, 10000, 20000, max_sweep_rate // 2]
  y_ticks = [1000, 5000, 10000, 20000, sample_rate//2]

  time, freq = _get_freq_ticks(max_sweep_rate, offset, sample_rate // 2)
  freq_x = [f if f in x_ticks and f <= max_sweep_rate // 2 else None for f in freq]
  freq_y = [f for f in freq if f >= 1000 and f in y_ticks and f <= sample_rate // 2]

  figure, axis = plt.subplots(1, 1)
  axis.specgram(waveform[0].numpy(), Fs=sample_rate)
  plt.xticks(time, freq_x)
  plt.yticks(freq_y, freq_y)
  axis.set_xlabel('Original Signal Frequency (Hz, log scale)')
  axis.set_ylabel('Waveform Frequency (Hz)')
  axis.xaxis.grid(True, alpha=0.67)
  axis.yaxis.grid(True, alpha=0.67)
  figure.suptitle(f'{title} (sample rate: {sample_rate} Hz)')
  plt.show(block=True)

def get_sine_sweep(sample_rate, offset=DEFAULT_OFFSET):
    max_sweep_rate = sample_rate
    freq = _get_log_freq(sample_rate, max_sweep_rate, offset)
    delta = 2 * math.pi * freq / sample_rate
    cummulative = torch.cumsum(delta, dim=0)
    signal = torch.sin(cummulative).unsqueeze(dim=0)
    return signal

def benchmark_resample(
    method,
    waveform,
    sample_rate,
    resample_rate,
    lowpass_filter_width=DEFAULT_LOWPASS_FILTER_WIDTH,
    rolloff=DEFAULT_ROLLOFF,
    resampling_method=DEFAULT_RESAMPLING_METHOD,
    beta=None,
    librosa_type=None,
    iters=5
):
  if method == "functional":
    begin = time.time()
    for _ in range(iters):
      F.resample(waveform, sample_rate, resample_rate, lowpass_filter_width=lowpass_filter_width,
                 rolloff=rolloff, resampling_method=resampling_method)
    elapsed = time.time() - begin
    return elapsed / iters
  elif method == "transforms":
    resampler = T.Resample(sample_rate, resample_rate, lowpass_filter_width=lowpass_filter_width,
                           rolloff=rolloff, resampling_method=resampling_method, dtype=waveform.dtype)
    begin = time.time()
    for _ in range(iters):
      resampler(waveform)
    elapsed = time.time() - begin
    return elapsed / iters
  elif method == "librosa":
    waveform_np = waveform.squeeze().numpy()
    begin = time.time()
    for _ in range(iters):
      librosa.resample(waveform_np, sample_rate, resample_rate, res_type=librosa_type)
    elapsed = time.time() - begin
    return elapsed / iters

In [ ]:
%matplotlib inline

I'll first look at an old-recording-method recording and a new-recording-method recording in each zone.

In [ ]:
old1_waveform, old1_sample_rate = torchaudio.load(OLD1_RECORDING_PATH)
new1_waveform, new1_sample_rate = torchaudio.load(NEW1_RECORDING_PATH)
old3_waveform, old3_sample_rate = torchaudio.load(OLD3_RECORDING_PATH)
new3_waveform, new3_sample_rate = torchaudio.load(NEW3_RECORDING_PATH)

In [ ]:
print(old3_waveform)
print(new3_waveform)

In [ ]:
print(torchaudio.info(OLD1_RECORDING_PATH))
print(torchaudio.info(NEW1_RECORDING_PATH))
print(torchaudio.info(OLD3_RECORDING_PATH))
print(torchaudio.info(NEW3_RECORDING_PATH))

In [ ]:
print(len(old1_waveform[0])/(old1_sample_rate*60))
print(len(new1_waveform[0])/(new1_sample_rate*60))
print(len(old3_waveform[0])/(old3_sample_rate*60))
print(len(new3_waveform[0])/(new3_sample_rate*60))

Not only are the recordings different lengths (~30 min vs 2.5 min), they're at different sample rates. The new hardware samples at 48 kHz while the old audio sampled at 22.05 kHz.

48 kHz is considered better bc 22 kHz causes aliasing down to 11 kHz, and humans can hear up to 15-20 kHz (https://www.ncbi.nlm.nih.gov/books/NBK10924/). However, it's a relatively small part of the perceptual range and probably doesn't have that much impact on speech recognition (https://www.frontiersin.org/articles/10.3389/fpsyg.2014.00587/full) (let alone whether it even comes through in BPC or is attenuated). Maybe the most significant aspect of this change, then, is the change in noise. 

In [ ]:
old1_waveform

In [ ]:
old1_waveform_silence = torch.unsqueeze(old1_waveform[0][0:old1_sample_rate*10],0)  # identified as silence/noise via listening
old1_waveform_speech = torch.unsqueeze(old1_waveform[0][old1_sample_rate*20:old1_sample_rate*30],0)  # identified as speech via listening
old3_waveform_silence = torch.unsqueeze(old3_waveform[0][0:old3_sample_rate*10],0)  # identified as silence/noise via listening
old3_waveform_speech = torch.unsqueeze(old3_waveform[0][old3_sample_rate*20:old3_sample_rate*30],0)  # identified as speech via listening

In [ ]:
print(len(old1_waveform_silence[0]))
print(len(old1_waveform_speech[0]))
print(len(old3_waveform_silence[0]))
print(len(old3_waveform_speech[0]))

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# plain waveform, silence
plot_waveform(old1_waveform_silence, old1_sample_rate,title="old zone 1 silence")
plot_waveform(old3_waveform_silence, old3_sample_rate,title="old zone 3 silence")

The magnitude of zone 3 peaks is 10x higher than zone 1 peaks.

Note the offset below 0 in Zone 1 and above 0 in Zone 3: what's that about?

It seems strange to have DC offset differing between zones, because the same acquisition equipment was used. Possibly it's a < 0.1 Hz component of the signal? Not sure what could cause that either...

In [ ]:
# measure offset
print(f'old zone 1 silence dc offset: {torch.mean(old1_waveform_silence)}')
print(f'old zone 3 silence dc offset: {torch.mean(old3_waveform_silence)}')

That's less offset than appears, actually...

In [ ]:
print(f'proportion points < 0: {len(old1_waveform_silence[old1_waveform_silence<0])/len(old1_waveform_silence[0])}')
print(f'proportion points > 0: {len(old1_waveform_silence[old1_waveform_silence>0])/len(old1_waveform_silence[0])}')

Most of the points must just be very close to zero.

In [ ]:
# plain waveform, speech
plot_waveform(old1_waveform_speech, old1_sample_rate,title="old zone 1 speech")
plot_waveform(old3_waveform_speech, old3_sample_rate,title="old zone 3 speech")

The most intuitive difference between speech and silence of course shows up: speech is far more energetic, with peaks 100x higher.

Zone 3 appears to be clipping a lot, if this plotting function worked correctly... but I'm not sure that's accurate.

In [ ]:
# specgram, silence
plot_specgram(old1_waveform_silence,old1_sample_rate,title="specgram - old zone 1 silence")
plot_specgram(old3_waveform_silence,old3_sample_rate,title="specgram - old zone 3 silence")

In [ ]:
# specgram, speech
plot_specgram(old1_waveform_speech,old1_sample_rate,title="specgram - old zone 1 speech")
plot_specgram(old3_waveform_speech,old3_sample_rate,title="specgram - old zone 3 speech")

Clear differences here. Unsurprisingly, the noise is far more uniform. The gap after 4 seconds in the speech spectrogram is a pause in talking. I believe the tall white noise (energy throughout the spectrum) spikes are the end-of-transmission sounds.

One interesting distinction here: the silence's energy is almost all under 6kHz, while the speech has significant spikes above (perhaps mostly due to static). Most of the speech energy is clearly still below 5kHz or so.

One other place I see a potentially interesting difference: right at the lowest frequencies, where there's a bit less energy in the speech sample but more energy in the silence sample.

In [ ]:
# spectrogram, silence

n_fft = 1024
win_length = None
hop_length = 512

# define transformation
spectrogram = T.Spectrogram(
    n_fft=n_fft,
    win_length=win_length,
    hop_length=hop_length,
    center=True,
    pad_mode="reflect",
    power=2.0,
)
# Perform transformation
spec1si = spectrogram(old1_waveform_silence)

print_stats(spec1si)
plot_spectrogram(spec1si[0], title='spectrogram - old zone 1 silence')

spec3si = spectrogram(old3_waveform_silence)

print_stats(spec3si)
plot_spectrogram(spec3si[0], title='spectrogram - old zone 3 silence')

In [ ]:
# spectrogram, speech 

spec1sp = spectrogram(old1_waveform_speech)

print_stats(spec1sp)
plot_spectrogram(spec1sp[0], title='spectrogram - old zone 1 speech')

spec3sp = spectrogram(old3_waveform_speech)

print_stats(spec3sp)
plot_spectrogram(spec3sp[0], title='spectrogram - old zone 3 speech')

Unsurprisingly, the "specgram" (uses matplotlib specgram function) and "spectrogram" (uses torchaudio) outputs are very similar. Of note are the mostly constant horizontal lines (component tones) of the noise.

In [ ]:
# mel spectrogram, silence

n_fft = 1024
win_length = None
hop_length = 512
n_mels = 128

mel_spectrogram_old = T.MelSpectrogram(
    sample_rate=old1_sample_rate,
    n_fft=n_fft,
    win_length=win_length,
    hop_length=hop_length,
    center=True,
    pad_mode="reflect",
    power=2.0,
    norm='slaney',
    onesided=True,
    n_mels=n_mels,
)

melspec1si = mel_spectrogram_old(old1_waveform_silence)
plot_spectrogram(
    melspec1si[0], title="MelSpectrogram - old zone 1 silence", ylabel='mel freq')

melspec1si = mel_spectrogram_old(old3_waveform_silence)
plot_spectrogram(
    melspec1si[0], title="MelSpectrogram - old zone 3 silence", ylabel='mel freq')

Something seems off in the mel frequency scale. Look into that.

In [ ]:
# mel spectrogram, speech

melspec1sp = mel_spectrogram_old(old1_waveform_speech)
plot_spectrogram(
    melspec1sp[0], title="MelSpectrogram - old zone 1 speech", ylabel='mel freq')

melspec3sp = mel_spectrogram_old(old3_waveform_speech)
plot_spectrogram(
    melspec3sp[0], title="MelSpectrogram - old zone 3 speech", ylabel='mel freq')

Again, check out that low to high frequency difference! If this held true, that would indicate the ratio of low to mid to high frequency carries a lot of info about silence vs nonsilence.

In [ ]:
# pitch, silence

print("pitch - old zone 1 silence")
pitch1si = F.detect_pitch_frequency(old1_waveform_silence, old1_sample_rate)
plot_pitch(old1_waveform_silence, old1_sample_rate, pitch1si)

print("pitch - old zone 3 silence")
pitch3si = F.detect_pitch_frequency(old3_waveform_silence, old3_sample_rate)
plot_pitch(old3_waveform_silence, old3_sample_rate, pitch1si)

In [ ]:
# pitch, speech

print("pitch - old zone 1 speech")
pitch1si = F.detect_pitch_frequency(old1_waveform_speech, old1_sample_rate)
plot_pitch(old1_waveform_speech, old1_sample_rate, pitch1si)

print("pitch - old zone 3 speech")
pitch3si = F.detect_pitch_frequency(old3_waveform_speech, old3_sample_rate)
plot_pitch(old3_waveform_speech, old3_sample_rate, pitch1si)

In [ ]:
# mfcc, silence

n_fft = 2048
win_length = None
hop_length = 512
n_mels = 256
n_mfcc = 256

mfcc_transform_old = T.MFCC(
    sample_rate=old1_sample_rate,
    n_mfcc=n_mfcc, melkwargs={'n_fft': n_fft, 'n_mels': n_mels, 'hop_length': hop_length})

mfcc1si = mfcc_transform_old(old1_waveform_silence)
plot_spectrogram(mfcc1si[0],title="mfcc - old zone 1 silence")

mfcc3si = mfcc_transform_old(old3_waveform_silence)
plot_spectrogram(mfcc3si[0],title="mfcc - old zone 3 silence")

In [ ]:
mfcc1sp = mfcc_transform_old(old1_waveform_speech)
plot_spectrogram(mfcc1sp[0],title="mfcc - old zone 1 speech")

mfcc3sp = mfcc_transform_old(old3_waveform_speech)
plot_spectrogram(mfcc3sp[0],title="mfcc - old zone 3 speech")

It seems clear that moving RMS on the plain waveform should generally be sufficient to detect audible sound vs silence. Moving ratios of bins of the spectrogram / mel spectrogram / MFCC can likely do the same, and perhaps even can distinguish some speech from other sounds. I will test the ratio approach here, basing it off of my observation that silence has relatively much stronger low-frequency vs middle-frequency components.

In [ ]:
# spec1si, spec3si, spec1sp, spec2sp
# melspec1si, melspec3si, melspec1sp, melspec3sp
# mfcc1si, mfcc3si, mfcc1sp, mfcc3sp

# all have length 431 (frames), but different # of frequency bins:

print(spec1si.shape)
print(melspec1si.shape)
print(mfcc1si.shape)

In [ ]:
# look at ratio of spectrogram bin mean values: 0-10:40-60

# silence
print(f"Zone 1 silence ratio: {torch.mean(spec1si[0,0:10,:])/torch.mean(spec1si[0,40:60,:])}")
print(f"Zone 3 silence ratio: {torch.mean(spec3si[0,0:10,:])/torch.mean(spec3si[0,40:60,:])}")

# speech (even including portions without speech)
print(f"Zone 1 speech ratio: {torch.mean(spec1sp[0,0:10,:])/torch.mean(spec1sp[0,40:60,:])}")
print(f"Zone 3 speech ratio: {torch.mean(spec3sp[0,0:10,:])/torch.mean(spec3sp[0,40:60,:])}")

In [ ]:
old1_waveform_mixed = torch.unsqueeze(old1_waveform[0][0:old1_sample_rate*30],0)  # first 10 seconds silence, last 10 speech
old3_waveform_mixed = torch.unsqueeze(old3_waveform[0][0:old3_sample_rate*30],0)  # first 10 seconds silence, last 10 speech

In [ ]:
# compute spectrograms for each
spec1mix = spectrogram(old1_waveform_mixed)
plot_spectrogram(spec1mix[0])
spec3mix = spectrogram(old3_waveform_mixed)
plot_spectrogram(spec3mix[0])

In [ ]:
# divide into 1-second slices along dimension 2
slice1mix = spec1mix.unfold(2,43,43)
# take mean of each bin for each second
binmeans1 = slice1mix[0,:,:,:].mean(dim=2,keepdim=True)[:,:,0]
# get bin ratio 0-10:40-60
binratio1 = binmeans1[0:10,:].mean(dim=0,keepdim=True)[0,:]/binmeans1[40:60,:].mean(dim=0,keepdim=True)[0,:]
plt.plot(binratio1)

# repeat for zone 3
slice3mix = spec3mix.unfold(2,43,43)
binmeans3 = slice3mix[0,:,:,:].mean(dim=2,keepdim=True)[:,:,0]
binratio3 = binmeans3[0:10,:].mean(dim=0,keepdim=True)[0,:]/binmeans3[40:60,:].mean(dim=0,keepdim=True)[0,:]
plt.plot(binratio3)

The above plot shows the second-by-second value of the ratio for each zone. We can see that the ratio of low- to middle-frequency components drops drastically right when speech begins in both recordings (remember that the first 20 seconds of each recording are silence, followed by roughly 10 seconds of speech). We can even see the momentary break in speech right at the start and end of the Zone 1 recording.

If we wanted, we could shrink the size of the moving-average window or overlap it with itself, etc. But I'm not sure that would add much as we want to retain some of the silence buffer around the moments of active speech, as those play an important role in conversation (and would make transcription more difficult if stripped).

An expansion on this approach would be to also look at the low/middle to high frequency ratio. This could be used to identify bursts of white noise such as radio squawks, which may have further downstream uses such as normalizing between zones. In general, by averaging a range of frequencies that are known to have high power in human voice, and averaging other frequencies that do not, and comparing their ratio, we may be able to distinguish speech from nonspeech sounds from silence. I am not sure that using mel-scale frequency would make this any more effective, and suspect that it could make it less effective.